In [1]:
#爬取网址：
#Request URL:https://www.lagou.com/jobs/positionAjax.json?needAdditionResult=false ‘显示频繁操作’
#Referer: https://www.lagou.com/zhaopin/Python/?labelWords=label
#Request Method: POST
#User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36

#思路整理：发送get请求，获取session-->更新session，发送post请求-->获取数据，储存-->数据多维度可视化



In [2]:
#代码：
#!/usr/bin/env python3
#encoding:utf-8
'''
Created on 19-5-05
@title:"
@author:Xusl
'''

import json
import requests
import xlwt
import time

In [13]:
#获取储存职位信息的json对象，遍历获得公司名、福利待遇、工作地点、学历要求、工作类型、发布时间、职位名称、薪资、工作年限
def get_json(url,datas):
    my_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
        'Referer': 'https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput=',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }
    time.sleep(5)
    #获取session
    ses = requests.Session()
    #更新
    ses.headers.update(my_headers)
    ses.get('https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput=')
    content = ses.post(url = url,data = datas)
    result = content.json()
    info = result['content']['positionResult']['result']
    info_list = []
    #-----------------------------------------------------------------------------------------
    #更新点：减少重复度，将分类添加至列表，遍历列表即可
    #-----------------------------------------------------------------------------------------
    classification_list = ['positionId','city','companyFullName','companyLabelList','district','education','firstType','formatCreateTime','positionName','salary','workYear']
    for job in info:
        information = []
        for classify in classification_list:
            information.append(job[classify])
        info_list.append(information)
        
    return info_list
        
        

In [14]:
#编写主函数
def main():
    page = int(input('Please input the total pages number you want to acquire'))
    info_result = []
    title = ['岗位id','城市','公司全名','福利待遇','工作地点','学历要求','工作类型','发布时间','职位名称','薪资','工作年限']
    info_result.append(title)
    for x in range(1,page+1):
        url = 'https://www.lagou.com/jobs/positionAjax.json?needAdditionResult=false'
        #更改点：'pn':x,   将x，更改为1，否则报错！！！！
        datas = {
            'first':'false',
            'pn':1,
            'kd':'python',
        }
        try:
            info = get_json(url,datas)
            info_result = info_result + info
            print('第%s页正常采集'%x)
        except Exception as msg:
            print('第%s页出现问题'%x)
            
        #创建workbook，即Excel
        workbook = xlwt.Workbook(encoding = 'utf-8')
        #创建表，第二参数用于确认同一个cell单元是否可以重设值
        worksheet = workbook.add_sheet('lagouzp',cell_overwrite_ok=True)
        for i,row in enumerate(info_result):
            for j,col in enumerate(row):
                worksheet.write(i,j,col)
        workbook.save('lagouzp.xls')
        
        
    

In [16]:
#执行函数
if __name__ == '__main__':
    main()

Please input the total pages number you want to acquire20
第1页正常采集
第2页正常采集
第3页正常采集
第4页正常采集
第5页正常采集
第6页正常采集
第7页正常采集
第8页正常采集
第9页正常采集
第10页正常采集
第11页正常采集
第12页正常采集
第13页正常采集
第14页正常采集
第15页正常采集
第16页正常采集
第17页正常采集
第18页正常采集
第19页正常采集
第20页正常采集


In [19]:
#使用pyecharts进行数据可视化
from pyecharts import Bar

city_nms_top10 = ['北京','上海','深圳','成都','杭州','广州','武汉','南京','苏州','郑州','天津','西安','东莞','珠海','合肥','厦门','宁波','南宁','重庆','佛山','大连','哈尔滨','长沙','福州','中山']
city_nums_top10 = [149,95,77,22,17,17,16,13,7,5,4,4,3,2,2,2,1,1,1,1,1,1,1,1,1]

bar = Bar('Python岗位','各城市数量')
bar.add('数量',city_nms_top10,city_nums_top10,is_more_utils = True)
bar.render('Python岗位各城市数量.html')

In [27]:
#地图分布展示
from pyecharts import Geo

city_nms_top10 = ['北京','上海','深圳','成都','杭州','广州','武汉','南京','苏州','郑州','天津','西安','东莞','珠海','合肥','厦门','宁波','南宁','重庆','佛山','大连','哈尔滨','长沙','福州','中山']
city_nums_top10 = [149,95,77,22,17,17,16,13,7,5,4,4,3,2,2,2,1,1,1,1,1,1,1,1,1]
city_datas = list(zip(city_nms_top10,city_nums_top10))

geo = Geo('Python岗位城市分布图','数据来源-拉钩',title_color = '#fff',
           title_pos = 'center',width = 1200,
           height = 600,background_color = '#404a59')
attr,value = geo.cast(city_datas)

geo.add('',attr,value,visual_range = [0,200],visual_text_color = '#fff',
        symbol_size = 15,is_visualmap = True)
geo.render('Python岗位城市分布地图_scatter.html')

geo = Geo('Python岗位城市分布地图','数据来源-拉钩',title_color = '#fff',
           title_pos = 'center',width = 1200,
           height = 600,background_color = '#404a59')
attr,value = geo.cast(city_datas)
geo.add('',attr,value,visual_range = [0,200],visual_text_color = '#fff',
        symbol_size = 15,is_visualmap = True)

geo.render('Python岗位城市分布地图_heatmap.html')


In [29]:
#各个城市招聘情况
from pyecharts import Pie

pie = Pie()
#只取前十位城市名称
pie.add('',city_nms_top10[:10],city_nums_top10[:10],is_label_show = True)

pie.render('Python岗位各城市分布饼图.html')